ADD COLAB BADGE (kida) 

In [ ]:
from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip -d dataset
!rm ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

In [23]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
import csv
import hashlib
import os
import pathlib
import pickle
import random
import re
import string
import timeit
import unicodedata
from collections import defaultdict
from itertools import combinations
from typing import Callable, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
import torch
import torch.nn.functional as F
from nltk.corpus import stopwords
from scipy.optimize import fsolve
from scipy.stats import kendalltau, spearmanr
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel

In [24]:
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_sm', exclude=['parser', 'ner'])
stops = set(stopwords.words('english'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Implementation

## Hash utils

In [3]:
def is_prime(n: int) -> bool:
    if n < 2:
        return False
    for i in range(2, int(np.sqrt(n))+1):
        if (n % i) == 0:
            return False
    return True

def find_closest_prime(n: int) -> int:
    """Finds the closest prime number higher than input."""
    while True:
        if is_prime(n):
            return n
        n += 1

def get_variable_length_hash(
    n_bits: int
) -> Callable[[str], int]:
    """Generates a hash function that takes a string
    as input and has 2 ** n_bits integer buckets.
    """
    def inner_f(s: str) -> int:
        binary_str = bin(
            int.from_bytes(
                hashlib.sha256(s.encode()).digest(), 
                'little'
            )
        )[-n_bits:]
        return int(binary_str, 2)
    return inner_f

class HashGenerator:
    """Generator of hash functions of the form:
            h(x) = (ax + b) mod c
    where x is a row number, a and b are random numbers
    smaller than the maximum row number and c is a prime
    number higher than the maximum row number.

    Note that a and b must be unique for a given signature
    matrix.

    This approach to hash function generation was suggested
    in [1].

    Parameters
    ----------
    num_rows : int
        Maximum number of rows of the characteristic matrix.

    References
    ----------
        [1] http://ethen8181.github.io/machine-learning/clustering_old/text_similarity/text_similarity.html

    """
    def __init__(
        self, 
        num_rows: int, 
    ) -> None:
        self.num_rows = num_rows
        self.prime = find_closest_prime(num_rows)
        self.a_set = set()
        self.b_set = set()

    def get_num_rows(self) -> int:
        return self.num_rows

    def next(self) -> Callable[[np.uint32], np.uint32]:
        """Returns a hash function that takes a row number 
        as input and returns another row number as output.
        """
        a = self._generate_coeff(self.a_set, self.num_rows)
        b = self._generate_coeff(self.b_set, self.num_rows)
        return lambda row: np.uint32((a * row + b) % self.prime)

    def _generate_coeff(
        self, 
        coeff_set: set[int],
        max_val: int
    ) -> int:
        while True:
            coeff = random.randint(1, max_val)
            if coeff not in coeff_set:
                coeff_set.add(coeff)
                return coeff

## Preprocessing utils

In [40]:
def normalize_white_space(doc: str) -> str:
    return " ".join(doc.split())

def remove_https(doc: str) -> str:
    return re.sub(r'https?://[^ ]+', '', doc)

def replace_chars(doc: str) -> str:
    return doc.replace('&amp;', ' and ')

def remove_non_ascii(doc: str) -> str:
    """Removes non ascii and non printable characters.
    We keep cyrillic characters due to the nature
    of the dataset.
    """
    cyr_chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"

    res = ""
    for c in doc:
        if (c.isascii() and c.isprintable()) \
            or (c in cyr_chars) or c.isspace():
            res += c
    return res

def strip_accents(doc: str) -> str:
    """Replaces words with accent with their 
    counterpart without accent. This also deals with 
    special characters such as 𝕒, 𝕕, 𝕖, 𝙖, 𝙘, 𝙙. 
    """
    return unicodedata.normalize('NFKD', doc)

def strip_punctuation(doc: str) -> str:
    return re.sub('[' + re.escape(string.punctuation) + ']+', '', doc)
    
def get_lemmatizer( 
    nlp: spacy.pipeline, 
    allow_stop_words: bool = False,
    allow_punct: bool = False,
    allow_numbers: bool = False
) -> Callable[[str], str]:
    """Generates a function that takes a string as
    input and returns the string sequence of lemmas
    in the input string. Optionally, the generated
    function removes stop words, punctuation and
    numbers.

    Note that numbers are tokens identified as such.
    For instance, '62,000' is a number, but 'T-72' is
    not.

    Parameters
    ----------
    nlp : spacy.pipeline
        Spacy object that carries out the lemmatization.
    
    allow_stop_words : bool
        Boolean value to filter or allow stop words.

    allow_punct : bool
        Boolean value to filter or allow punctuation.
    
    allow_numbers : bool
        Boolean value to filter or allow numbers.

    Returns
    -------
    The generated function. 
    """
    def inner_f(doc: str) -> str:
        return ' '.join(
            [
                token.lemma_
                for token in nlp(doc)
                if (not token.is_stop or allow_stop_words) \
                    and (not token.is_punct or allow_punct) \
                    and (token.pos_ != 'NUM' or allow_numbers) \
                    and (not token.pos_ == 'X')
            ]
        )
    return inner_f

def remove_handles(doc: str) -> str:
    return re.sub(r'@\w+', '', doc)

def remove_short(n: int) -> Callable[[str], str]:
    def inner_f(doc: str) -> str:
        if len(doc) < n:
            return ''
        else:
            return doc

    return inner_f

def get_stopwords_remover(
    stops: list[str]
) -> Callable[[str], str]:
    def inner_f(doc: str) -> str:
        return ' '.join(
            [
                token for token in doc.split()
                if token not in stops
            ]
        )
    return inner_f

## Model

In [5]:
def jaccard_similarity(
    x: np.ndarray, 
    y: np.ndarray
) -> float:
    numerator = len(set(x).intersection(set(y)))
    denominator = len(set(x).union(set(y)))
    return numerator / denominator

class LSHModel:
    """Implementation of LSH model that finds similar pairs
    of documents encoded as k-gram shingles.

    Parameters
    ----------
    k : int
        Number of characters in each k-gram.

    threshold : float
        The similarity value required to consider a
        pair as similar.

    num_hashes : int
        Number of hash functions used to generate the
        signature matrix.

    shingle_hash_bits : int
        Determines the number of buckets of the hash
        function that maps each shingle to an integer.

    track_shingles : bool
        Flag to keep track of the number of different
        shingles found in the corpus, as well as the
        number of different characters in the shingles.

    checkpoint_path : Optional[str]
        Path to save and load the state of the model.

    Exceptions
    ----------
    ValueError
        If the number of hash functions is higher than the
        number of rows of the characteristic matrix (which is
        also the number of shingles). This is due to the fact
        that the the coefficients 'a' and 'b' of the hash
        functions generated by HashGenerator need to be unique
        within the given signature matrix.
    """
    def __init__(
        self,
        k: int,
        threshold: float,
        num_hashes: int,
        shingle_hash_bits: int,
        track_shingles: bool = False,
        checkpoint_path: Optional[str] = None
    ) -> None:
        self.k = k
        self.threshold = threshold
        self.num_hashes = num_hashes
        self.shingle_set = set()
        self.char_set = set()
        self.shingle_hash_bits = shingle_hash_bits
        self.shingle_hash = get_variable_length_hash(
            shingle_hash_bits
        )
        self.num_shingles = 2 ** shingle_hash_bits
        self.track_shingles = track_shingles
        self.checkpoint_path = checkpoint_path
        self.num_docs = 0
        self.docs_dict = dict()
        self.signature = None
        self.candidate_pairs = set()
        self.fp_pairs = set()
        self.similar_pairs = set()
        self.b = -1
        self.r = -1
        self.sig_idx = -1

        if self.num_hashes > self.num_shingles:
            raise ValueError(
                f"Number of hash functions must be lower than "
                f"or equal to the number of shingles. Found "
                f"{self.num_hashes} hash functions and "
                f"{self.num_shingles} shingles."
            )

    def load_checkpoint(
        self,
        checkpoint_path: Optional[str] = None
    ) -> None:
        if checkpoint_path is not None:
            self.checkpoint_path = checkpoint_path
        if self.checkpoint_path is None:
            raise ValueError(
                "Checkpoint path not found"
            )
        else:
            tup_ls = [
                (f'{self.checkpoint_path}/docs_dict.npy', 'docs_dict'),
                (f'{self.checkpoint_path}/shingle_set.npy', 'shingle_set'),
                (f'{self.checkpoint_path}/char_set.npy', 'char_set'),
                (f'{self.checkpoint_path}/signature.npy', 'signature'),
                (f'{self.checkpoint_path}/sig_idx.npy', 'sig_idx'),
                (f'{self.checkpoint_path}/candidate_pairs.npy', 'candidate_pairs'),
                (f'{self.checkpoint_path}/fp_pairs.npy', 'fp_pairs'),
                (f'{self.checkpoint_path}/similar_pairs.npy', 'similar_pairs')
            ]

            for file_path, attr in tup_ls:
                if os.path.isfile(file_path):
                    if attr in ['signature']:
                        setattr(
                            self, 
                            attr, 
                            np.load(file_path, allow_pickle=True)
                        )
                    else:
                        setattr(
                            self, 
                            attr, 
                            np.load(file_path, allow_pickle=True).item()
                        )
                        
    def save_checkpoint(
        self,
        checkpoint_path: Optional[str] = None
    ) -> None:
        if checkpoint_path is not None:
            self.checkpoint_path = checkpoint_path
        if self.checkpoint_path is None:
            raise ValueError(
                "Checkpoint path not found"
            )
        else:
            os.makedirs(self.checkpoint_path, exist_ok=True)

            tup_ls = [
                (f'{self.checkpoint_path}/docs_dict.npy', self.docs_dict),
                (f'{self.checkpoint_path}/shingle_set.npy', self.shingle_set),
                (f'{self.checkpoint_path}/char_set.npy', self.char_set),
                (f'{self.checkpoint_path}/signature.npy', self.signature),
                (f'{self.checkpoint_path}/sig_idx.npy', self.sig_idx),
                (f'{self.checkpoint_path}/candidate_pairs.npy', self.candidate_pairs),
                (f'{self.checkpoint_path}/fp_pairs.npy', self.fp_pairs),
                (f'{self.checkpoint_path}/similar_pairs.npy', self.similar_pairs)
            ]

            for file_path, val in tup_ls:
                np.save(file_path, val)

    def add_document(
        self, 
        doc: str,
        preprocessing_pipeline: Optional[list[Callable[[str], str]]] = None
    ) -> None:
        """Creates shingles from the document given in input and
        adds those shingles to the model. Optionally, the document
        is preprocessed with a number of functions given in a 
        pipeline.

        Parameters
        ----------
        doc : str
            String document to be processed.

        preprocessing_pipeline : Optional[list[Callable[[str], str]]]
            List of functions that take a string and return a string.
            This is used to filter stop words, apply lemmatization, etc.
        """
        if preprocessing_pipeline is not None:
            for f in preprocessing_pipeline:
                doc = f(doc)

        # print(self.num_docs, doc)
        # if '_' in doc:
        #     print((self.num_docs, doc))
        
        shingles = self._create_shingles(
            doc, 
            self.k,
            self.track_shingles,
            self.shingle_hash
        )

        self.docs_dict[self.num_docs] = shingles
        self.num_docs += 1

    def get_similar_pairs(
        self,
        checkpoint_path: Optional[str] = None,
        checkpoint_freq: int = 10000
    ) -> set[tuple[tuple[int, int], float]]:
        """Returns the pairs having an approximated similarity 
        higher than a fixed threshold. The pairs are provided as 
        a set of tuples containing the indices of the documents and
        their similarity value. 
        
        The approximated similarity measure is the Jaccard
        similarity.

        This function also saves the false positive pairs identified
        after double-checking the signature matrix.

        Parameters
        ----------
        checkpoint_path : Optional[str]
            Path to save and load the state of the model. This is used
            when building the signature matrix.

        checkpoint_freq : int
            Frequency with which the state of the model is saved.
        
        Returns
        -------
        The set of pairs approximately similar, alongside their 
        similarity value.
        """
        hg = HashGenerator(self.num_shingles)
        hash_functions = [
            hg.next()
            for _ in range(self.num_hashes)
        ]
        self.signature = self._build_signature(
            self.docs_dict,
            self.num_shingles,
            hash_functions,
            checkpoint_path,
            checkpoint_freq
        )
        self.b, self.r = self._find_lsh_params(
            self.threshold,
            self.num_hashes
        )
        self.candidate_pairs = self._lsh(
            self.signature,
            self.b
        )
        self.similar_pairs, self.fp_pairs = \
            self._check_threshold_on_signature(
                self.candidate_pairs,
                self.signature,
                self.threshold
            )
        return self.similar_pairs

    def _create_shingles(
        self,
        doc: str, 
        k: int,
        track_shingles: bool, 
        hash_f: Callable[[str], int]
    ) -> np.ndarray:
        res = []

        for i in range(len(doc[:-k+1])):
            shingle = doc[i:i+k]
            if track_shingles:
                self.shingle_set.add(shingle)
                self.char_set = self.char_set.union(
                    set(shingle)
                ) 
            res.append(hash_f(shingle))

        return np.unique(res).astype(np.uint32)

    def _build_signature(
        self,
        docs_dict: dict[int, np.ndarray],
        num_rows: int, 
        hash_functions: list[Callable[[np.uint32], np.uint32]],
        checkpoint_path: Optional[str] = None,
        checkpoint_freq: int = 10000
    ) -> np.ndarray:
        if checkpoint_path is not None:
            self.checkpoint_path = checkpoint_path
        if self.checkpoint_path is not None:
            os.makedirs(self.checkpoint_path, exist_ok=True)
        
        sig_path = f'{self.checkpoint_path}/temp_signature.npy'
        sig_idx_path = f'{self.checkpoint_path}/temp_sig_idx.npy'
        
        if self.checkpoint_path is not None and \
            os.path.isfile(sig_path) and \
            os.path.isfile(sig_idx_path):
                signature = np.load(sig_path, allow_pickle=True)
                self.sig_idx = np.load(
                    sig_idx_path, 
                    allow_pickle=True
                ).item()
                print(f"Loaded signature from row {self.sig_idx}")
        else:
            signature = np.full(
                (len(hash_functions), len(docs_dict)), 
                fill_value=np.inf
            )
            self.sig_idx = -1

        for r in tqdm(
            range(0, num_rows),
            total=num_rows,
            desc='[Signature matrix] row number',
            leave=False
        ):
            if r < self.sig_idx:
                continue

            hash_values = [
                f(r)
                for f in hash_functions
            ]
            for c, shingles in enumerate(docs_dict.values()):
                if r in shingles:
                    for i, hash_val in enumerate(hash_values):
                        if hash_val < signature[i,c]:
                            signature[i,c] = hash_val

            self.sig_idx = r
            if (self.sig_idx % checkpoint_freq == 0) and \
                self.checkpoint_path is not None:
                np.save(sig_path, signature)
                np.save(sig_idx_path, self.sig_idx)

        if self.checkpoint_path is not None:
            np.save(sig_path, signature)
            np.save(sig_idx_path, self.sig_idx)
        
        return signature.astype(np.uint32)

    def _find_lsh_params(self, t: int, n: int) -> tuple[int]:
        """Note that a lower b means that two items must match 
        a higher number of rows. By taking the floor of b, we 
        favor more similar pairs.

        Sympy did not always find a solution.
        """
        def equations(vars):
            b, r = vars
            eq1 = t - (1 / b) ** (1 / r)
            eq2 = n - b * r
            return [eq1, eq2]

        b, r =  fsolve(equations, (1, 1))
        b = np.floor(b)
        r = n // b
        return int(b), int(r)

    def _lsh(
        self, 
        signature: np.ndarray, 
        b: int
    ) -> set[tuple[int, int]]:
        candidate_pairs = set()
        
        bands = np.array_split(signature, b)

        for band in tqdm(
            bands,
            total=len(bands),
            desc='[LSH] band number',
            leave=False
        ):
            # column tuple -> list of column indices having that tuple
            same_columns = defaultdict(list) 
            
            for c in range(band.shape[1]):
                column = band[:,c]
                str_column = ''.join([str(num) for num in column])
                same_columns[hash(str_column)].append(c)

            for k in list(same_columns.keys()):
                if len(same_columns[k]) < 2:
                    del same_columns[k]

            for values in same_columns.values():
                indices = range(len(values))
                for i in indices:
                    for j in range(i+1, len(values)):
                        candidate_pairs.add((values[i], values[j]))

        return candidate_pairs

    def _check_threshold_on_signature(
        self, 
        candidate_pairs: list[tuple[int, int]], 
        signature: np.ndarray, 
        t: float
    ) -> tuple[set[tuple[tuple[int, int], float]]]:
        similar_pairs = set()
        false_positive_pairs = set()

        for (x, y) in tqdm(
            candidate_pairs,
            total=len(candidate_pairs),
            desc='[Threshold check] pair number',
            leave=False
        ):
            x_col = signature[:,x]
            y_col = signature[:,y]
            similarity = sum(x_col == y_col) / signature.shape[0]
            tup = ((x, y), similarity)
            if similarity >= t:
                similar_pairs.add(tup)
            else:
                false_positive_pairs.add(tup)

        return similar_pairs, false_positive_pairs

    def check_threshold_on_cm(
        self
    ) -> tuple[set[tuple[tuple[int, int], float]]]:
        """Returns two sets of pairs. The first is the set
        of similar pairs obtained after checking the
        pairs returned by the LSH procedure against the actual 
        Jaccard similarity computed from the characteristic matrix.
        The second is the set of false positive pairs identified
        after the double-check against the characteristic matrix.
        """
        similar_pairs = set()
        false_positive_pairs = set()

        for ((x, y), _) in self.similar_pairs:
            similarity = jaccard_similarity(
                self.docs_dict[x], 
                self.docs_dict[y]
            )
            tup = ((x, y), similarity)
            if similarity >= self.threshold:
                similar_pairs.add(tup)
            else:
                false_positive_pairs.add(tup)

        return similar_pairs, false_positive_pairs

    def get_shingle_set(self) -> set[int]:
        return self.shingle_set

    def get_char_set(self) -> set[str]:
        return self.char_set

    def get_docs_dict(self) -> dict[int, np.ndarray]:
        return self.docs_dict

## General utils

In [6]:
def mean_absolute_error(
    x: list, 
    y: list
) -> float:
    return sum(
        [np.abs(val2 - val1) for val1, val2 in zip(x, y)]
    ) / (len(x) or 1e-10) # to avoid division by zero

def evaluate_on_cm(
    sig_dict: dict[tuple[int, int], float], 
    cm_dict: dict[tuple[int, int], float]
) -> tuple[int, float]:
    """Evaluates the model performance by computing
    the number of false positive pairs and the
    mean absolute error (MAE) against the characteristic
    matrix.

    Parameters
    ----------
    sig_dict : dict[tuple[int, int], float]
        Dictionary that maps each similar pair to the
        corresponding similarity value obtained as
        estimation from the signature matrix.

    cm_dict : dict[tuple[int, int], float]
        Dictionary that maps each similar pair to the
        corresponding similarity value obtained by
        computing the Jaccard similarity on the 
        characteristic matrix.
    
    Returns
    -------
    The number of false positive pairs and the MAE.
    """
    common = set(sig_dict).intersection(set(cm_dict))
    num_wrong = len(sig_dict) - len(common)

    sig_values = []
    cm_values = []

    for pair in common:
        sig_values.append(sig_dict[pair])
        cm_values.append(cm_dict[pair])

    return num_wrong, \
        mean_absolute_error(sig_values, cm_values) 

def train_model(
    model: LSHModel, 
    data_path: str, 
    num_docs: int,
    num_blocks: int = 20,
    verbose: bool = False,
    filtering_pipeline: Optional[list[Callable[[str], str]]] = None, 
    preprocessing_pipeline: Optional[list[Callable[[str], str]]] = None  
) -> LSHModel:
    """Trains the model on a given number of documents
    taken from a provided dataset. Training here means
    adding the shingles of the documents to the model.

    Parameters
    ----------
    model : LSHModel
        The model to be trained.

    data_path : str
        The path where the files of the dataset are
        stored.

    num_docs : int
        The number of documents on which the model
        will be trained.

    num_blocks : int
        Number of files to read in chunks.

    verbose : bool
        Flag that determines whether to print 
        information about the processing.

    filtering_pipeline : Optional[list[Callable[[str], str]]]
        List of functions that take a string and return a string.
        This is used on the text field of the dataframe, before
        feeding the data to the model and will be used to 
        determine duplicates to drop.

    preprocessing_pipeline : Optional[list[Callable[[str], str]]]
        List of functions that take a string and return a string.
        This is used to preprocess documents being added to 
        the model.
    
    Returns
    -------
    The trained model.
    """
    files = []

    for name in os.listdir(data_path):
        full_path = os.path.join(data_path, name)
        if os.path.isfile(full_path):
            files.append(full_path)

    files = np.array(files)
    duplicates = 0
    count = num_docs

    with tqdm(
        total=num_docs,
        desc='Adding documents to model',
        leave=False
    ) as pbar:

        files_blocks = np.array_split(files, num_blocks)

        for file_block in files_blocks:

            dfs = []

            for file in file_block:
                if count == 0:
                    break

                if verbose:
                    print(f'Reading file {file}')
                file_df = pd.read_csv(
                    file, 
                    compression='gzip', 
                    index_col=0,
                    encoding='utf-8', 
                    quoting=csv.QUOTE_ALL,
                    low_memory=False
                )
                file_df = file_df[file_df['language'] == 'en']
                dfs.append(file_df)

            df = pd.concat(dfs).reset_index()

            if filtering_pipeline is not None:
                for filter_f in filtering_pipeline:
                    df['text'] = df['text'].apply(filter_f)

            df_unique = df.drop_duplicates(subset=['text'])
            df_unique = df_unique[df_unique['text'] != '']
            duplicates += len(df) - len(df_unique)

            for index, row in tqdm(
                df_unique.iterrows(),
                total=len(df_unique),
                desc='Reading file',
                leave=False
            ):
                text = row['text']
                model.add_document(
                    text,
                    preprocessing_pipeline
                )
                
                count -= 1
                pbar.update(1)
                if count == 0:
                    if verbose:       
                        print(f'Filtered {duplicates} rows in files, kept {len(df_unique)}')
                    return model

def get_text(
    idx_ls: list[int], 
    data_path: str,
    num_blocks: int = 20,
    filtering_pipeline: Optional[list[Callable[[str], str]]] = None
) -> list[tuple[int, str]]:
    """Returns a list containing the original texts
    from the dataset (before the preprocessing) alongside
    their indices.

    Parameters
    ----------
    idx_ls : list[int]
        The list of the indices of the documents to 
        be retrieved.

    data_path : str
        The path where the files of the dataset are
        stored.

    num_blocks : int
        Number of files to read in chunks.

    filtering_pipeline : Optional[list[Callable[[str], str]]]
        List of functions that take a string and return a string.
        This is used on the text field of the dataframe, before
        feeding the data to the model and will be used to 
        determine duplicates to drop.
    
    Returns
    -------
    Tuples containing the indices of the documents and their
    original text.
    """
    max_idx = max(idx_ls)
    result = []
    
    files = []

    for name in os.listdir(data_path):
        full_path = os.path.join(data_path, name)
        if os.path.isfile(full_path):
            files.append(full_path)
    
    files = np.array(files)
    count = 0

    files_blocks = np.array_split(files, num_blocks)

    for file_block in tqdm(
        files_blocks,
        total=len(files_blocks),
        desc='File block',
        leave=False
    ):

        dfs = []

        for file in file_block:
            print(f'Reading file {file}')

            file_df = pd.read_csv(
                file, 
                compression='gzip', 
                index_col=0,
                encoding='utf-8', 
                quoting=csv.QUOTE_ALL,
                low_memory=False
            )
            file_df = file_df[file_df['language'] == 'en']
            dfs.append(file_df)

        df = pd.concat(dfs).reset_index()
        df_to_filter = df.copy()

        if filtering_pipeline is not None:
            for filter_f in filtering_pipeline:
                df_to_filter['text'] = df_to_filter['text'].apply(filter_f)

        df_unique = df_to_filter.drop_duplicates(subset=['text'])
        df_unique = df_unique[df_unique['text'] != '']
        df_filtered = df.iloc[df_unique.index]

        for index, row in tqdm(
            df_filtered.iterrows(),
            total=len(df_filtered),
            desc='Reading file',
            leave=False
        ):
            if count in idx_ls:
                result.append((count, row['text']))
            if count == max_idx:
                return result
            count += 1

def mean_pooling(
    model_output: torch.Tensor, 
    attn_mask: torch.Tensor
) -> torch.Tensor:
    """Returns the mean of the embeddings taken from 
    the last layer of the model, in order to give 
    a single embedding for each document. The mean
    is weighted with the attention mask, so that 
    the padding and control tokens added by the model
    are not considered in the mean.

    Parameters
    ----------
    model_output : torch.Tensor
        Embeddings for all the documents.

    attn_mask : torch.Tensor
        The attention mask of the model for all the
        documents.
    
    Returns
    -------
    The weighted mean embedding for each document. 
    """
    token_embeddings = model_output['last_hidden_state']

    # attn_mask shape: [13, 512] -> [13, 512, 768]
    expanded_attn_mask = attn_mask.unsqueeze(-1).expand_as(token_embeddings)

    # * or torch.mul: out_i = input_i x other_i 
    # might use torch.clamp to avoid dividing by 0
    return torch.sum(
        token_embeddings * expanded_attn_mask, 1
    ) / expanded_attn_mask.sum(1)

def torch_cosine_similarity(x, y):
    return torch.matmul(
        F.normalize(x, dim=-1), 
        F.normalize(y, dim=-1)
    )

# Experiments

In [42]:
if os.path.isdir(r'e:\datasets\ukraine'):
    DATA_PATH = r'e:\datasets\ukraine'
else:
    DATA_PATH = os.path.join(os.getcwd(), 'dataset')

os.makedirs('img', exist_ok=True)

In [43]:
filtering_pipeline = [
    remove_https,
    remove_handles,
    strip_accents,
    replace_chars,
    str.lower,
    remove_non_ascii,
    strip_punctuation,
    get_stopwords_remover(stops),
    normalize_white_space,
    remove_short(100)
]

preprocessing_pipeline = [
    get_lemmatizer(
        nlp,
        allow_numbers=True
    ),
    strip_punctuation,
    normalize_white_space
]

## Shingle and character number growth

In [ ]:
results = dict()

for k in [3, 4, 5, 6]:
    results[k] = {
        'docs': [],
        'characters': [],
        'shingles': [],
        'avg_shingles': []
    }

    for num_docs in [
        10, 100, 1000, 10000, 
        20000, 30000, 50000,
        70000, 100000, 150000,
        200000
    ]:
        ckpt_path = f'checkpoints/k{k}_d{num_docs}'
        model = LSHModel(
            k=k,
            threshold=0.1,
            num_hashes=100,
            shingle_hash_bits=16,
            track_shingles=True,
            checkpoint_path=ckpt_path
        )

        if os.path.isdir(ckpt_path) and \
            len(os.listdir(ckpt_path)) > 0:
            model.load_checkpoint()
        else:
            model = train_model(
                model=model, 
                data_path=DATA_PATH,
                num_docs=num_docs,
                verbose=False,
                filtering_pipeline=filtering_pipeline,
                preprocessing_pipeline=preprocessing_pipeline,
            )
            model.save_checkpoint()

        results[k]['docs'].append(num_docs)
        results[k]['characters'].append(len(model.get_char_set()))
        results[k]['shingles'].append(len(model.get_shingle_set()))

        docs_dict = model.get_docs_dict()
        avg_shingles = np.mean(
            [
                len(doc_shingles) 
                for doc_shingles in docs_dict.values()
            ]
        )
        results[k]['avg_shingles'].append(avg_shingles)

        print(
            f'[{k} k, {num_docs} docs]:\n'
            f'\t{len(model.get_char_set())} characters\n'
            f'\t{len(model.get_shingle_set())} shingles\n'
            f'\t{avg_shingles} avg shingles\n'
        )

In [ ]:
for k in [3, 4, 5, 6]:
    plt.plot(
        results[k]['docs'], 
        results[k]['shingles'],
        label=f'k = {k}'
    )
plt.xticks([0, 50000, 100000, 150000, 200000])
plt.xlabel('Number of documents')
plt.ylabel('Number of shingles')
plt.title('Shingles growth')
plt.legend(loc='best')
plt.savefig('img/shingles_growth.png', dpi=300)

In [ ]:
plt.plot(
    results[5]['docs'], 
    results[5]['characters']
)
plt.xticks([0, 50000, 100000, 150000, 200000])
plt.xlabel('Number of documents')
plt.ylabel('Number of characters')
plt.title('Characters growth')
plt.savefig('img/char_growth.png', dpi=300)

## Number of hash bits

In [ ]:
results = dict()

for k in [3, 4, 5]:
    ls = []

    for n_bits in [12, 14, 16, 18, 19, 20, 22]:
        ckpt_path = f'checkpoints/k{k}_n_bits{n_bits}'
        time_path = f'{ckpt_path}/time.npy'

        model = LSHModel(
            k=k,
            threshold=0.1,
            num_hashes=100,
            shingle_hash_bits=n_bits,
            track_shingles=True,
            checkpoint_path=ckpt_path
        )

        if os.path.isdir(ckpt_path) and \
            len(os.listdir(ckpt_path)) > 0:
            model.load_checkpoint()

            time_delta = np.load(
                f'{ckpt_path}/time.npy', 
                allow_pickle=True
            )

            sig_tp = dict(model.get_similar_pairs())
            
        else:
            start_time = timeit.default_timer()

            model = train_model(
                model=model, 
                data_path=DATA_PATH,
                num_docs=100,
                verbose=False,
                filtering_pipeline=filtering_pipeline,
                preprocessing_pipeline=preprocessing_pipeline,
            )
            model.save_checkpoint()

            sig_tp = dict(model.get_similar_pairs())

            end_time = timeit.default_timer()
            time_delta = end_time - start_time
            np.save(f'{ckpt_path}/time.npy', time_delta)

        cm_tp, _ = model.check_threshold_on_cm()
        cm_tp = dict(cm_tp)
        num_wrong, mae = evaluate_on_cm(sig_tp, cm_tp)
        correct = len(sig_tp) - num_wrong
        ratio = correct / len(sig_tp)

        ls.append(
            (
                n_bits, 
                time_delta,
                len(sig_tp),
                correct, 
                num_wrong,
                ratio,
                mae
            )
        )

        print(
            f'[k {k}, {n_bits} bits]:\n'
            f'\t{time_delta} seconds\n'
            f'\t{num_wrong} wrong out of {len(sig_tp)} ({ratio} Prec.) (0.1 t)\n'
            f'\t{mae} MAE\n'
        )
    
    results[k] = pd.DataFrame(
        ls,
        columns=[
            'Hash bits', 
            'Time delta (s)', 
            'Predicted pairs',
            'Correct pairs (TP)', 
            'Wrong pairs (FP)', 
            'Correct ratio (Prec.)',
            'MAE'
        ]
    ).set_index('Hash bits')

In [ ]:
for k in [3, 4, 5]:
    print(results[k].round(3)) 

## Threshold choice

In [ ]:
results = dict()

for k in [3, 4, 5]:
    ls = []

    for t in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.5]:
        ckpt_path = f'checkpoints/k{k}_t{t}'

        model = LSHModel(
            k=k,
            threshold=t,
            num_hashes=100,
            shingle_hash_bits=16,
            track_shingles=True,
            checkpoint_path=ckpt_path
        )

        if os.path.isdir(ckpt_path) and \
            len(os.listdir(ckpt_path)) > 0:
            model.load_checkpoint()
            sig_tp = dict(model.get_similar_pairs())
            
        else:
            model = train_model(
                model=model, 
                data_path=DATA_PATH,
                num_docs=100,
                verbose=False,
                filtering_pipeline=filtering_pipeline,
                preprocessing_pipeline=preprocessing_pipeline,
            )
            model.save_checkpoint()
            sig_tp = dict(model.get_similar_pairs())

        cm_tp, _ = model.check_threshold_on_cm()
        cm_tp = dict(cm_tp)
        num_wrong, mae = evaluate_on_cm(sig_tp, cm_tp)
        correct = len(sig_tp) - num_wrong
        ratio = correct / (len(sig_tp) or 1e-10)

        ls.append(
            (
                t,
                len(sig_tp),
                correct, 
                num_wrong,
                ratio,
                mae
            )
        )

        print(
            f'[k {k}, {t} threshold]:\n'
            f'\t{num_wrong} wrong out of {len(sig_tp)} ({ratio} Prec.)\n'
            f'\t{mae} MAE\n'
        )
    
    results[k] = pd.DataFrame(
        ls,
        columns=[
            'Threshold', 
            'Predicted pairs',
            'Correct pairs (TP)', 
            'Wrong pairs (FP)', 
            'Correct ratio (Prec.)',
            'MAE'
        ]
    ).set_index('Threshold')

In [ ]:
for k in [3, 4, 5]:
    print(results[k].round(3))

## Number of hash functions

In [ ]:
results = dict()

for k in [3, 4, 5]:
    ls = []

    for num_hashes in [20, 100, 200, 500, 1000]:
        ckpt_path = f'checkpoints/k{k}_n_hash{num_hashes}'
        time_path = f'{ckpt_path}/time.npy'

        model = LSHModel(
            k=k,
            threshold=0.1,
            num_hashes=num_hashes,
            shingle_hash_bits=16,
            track_shingles=True,
            checkpoint_path=ckpt_path
        )

        if os.path.isdir(ckpt_path) and \
            len(os.listdir(ckpt_path)) > 0:
            model.load_checkpoint()
            sig_tp = dict(model.get_similar_pairs())
            time_delta = np.load(
                f'{ckpt_path}/time.npy', 
                allow_pickle=True
            )
            
        else:
            start_time = timeit.default_timer()

            model = train_model(
                model=model, 
                data_path=DATA_PATH,
                num_docs=100,
                verbose=False,
                filtering_pipeline=filtering_pipeline,
                preprocessing_pipeline=preprocessing_pipeline,
            )
            model.save_checkpoint()

            sig_tp = dict(model.get_similar_pairs())

            end_time = timeit.default_timer()
            time_delta = end_time - start_time
            np.save(f'{ckpt_path}/time.npy', time_delta)

        cm_tp, _ = model.check_threshold_on_cm()
        cm_tp = dict(cm_tp)
        num_wrong, mae = evaluate_on_cm(sig_tp, cm_tp)
        correct = len(sig_tp) - num_wrong
        ratio = correct / (len(sig_tp) or 1e-10)

        ls.append(
            (
                num_hashes,
                time_delta,
                len(sig_tp),
                correct, 
                num_wrong,
                ratio,
                mae
            )
        )

        print(
            f'[k {k}, {num_hashes} hashes]:\n'
            f'\t{time_delta} seconds\n'
            f'\t{num_wrong} wrong out of {len(sig_tp)} ({ratio} Prec.)\n'
            f'\t{mae} MAE\n'
        )
    
    results[k] = pd.DataFrame(
        ls,
        columns=[
            'Num hashes', 
            'Time delta (s)',
            'Predicted pairs',
            'Correct pairs (TP)', 
            'Wrong pairs (FP)', 
            'Correct ratio (Prec.)',
            'MAE'
        ]
    ).set_index('Num hashes')

## 100k Tweets comparison

### Train LSH model

MORE THAN ONE MODEL?

In [12]:
ckpt_path = f'checkpoints/d100k/k4_t0.2_n_hashes200_n_bits18'
model = LSHModel(
    k=4,
    threshold=0.4,
    num_hashes=200,
    shingle_hash_bits=18,
    track_shingles=True,
    checkpoint_path=ckpt_path
)
model = train_model(
    model=model, 
    data_path=DATA_PATH,
    num_docs=100000,
    verbose=True,
    filtering_pipeline=filtering_pipeline,
    preprocessing_pipeline=preprocessing_pipeline
)

Adding documents to model:   0%|          | 0/100000 [00:00<?, ?it/s]

Reading file e:\datasets\ukraine\0401_UkraineCombinedTweetsDeduped.csv.gzip
Reading file e:\datasets\ukraine\0402_UkraineCombinedTweetsDeduped.csv.gzip
Reading file e:\datasets\ukraine\0403_UkraineCombinedTweetsDeduped.csv.gzip
Reading file e:\datasets\ukraine\0404_UkraineCombinedTweetsDeduped.csv.gzip
Reading file e:\datasets\ukraine\0405_UkraineCombinedTweetsDeduped.csv.gzip
Reading file e:\datasets\ukraine\0406_UkraineCombinedTweetsDeduped.csv.gzip


KeyboardInterrupt: 

In [ ]:
similar_pairs = model.get_similar_pairs()

In [ ]:
model.docs_dict

In [ ]:
get_text((14307, 21787), DATA_PATH, filtering_pipeline=filtering_pipeline)

In [ ]:
ss = [
    '@For_Freedom_Rus @MaajidNawaz Please, explain why they built Nazi collaborator monuments in #Ukraine?\n\nhttps://t.co/8T4A30abWx\n\nUkraine Neo-Nazis Infiltrate EVERY LEVEL Of Military &amp; Government\n\nJimmy Dore: https://t.co/I9VPYQvSxi',
    '@SecYellen Please, explain why they built Nazi collaborator monuments in #Ukraine?\n\nhttps://t.co/8T4A2ZSB4Z\n\nUkraine Neo-Nazis Infiltrate EVERY LEVEL Of Military &amp; Government\n\nby Jimmy Dore: https://t.co/I9VPYQehFK https://t.co/llS2RQJG0h'
]
for s in ss:
    for f in filtering_pipeline:
        s = f(s)
    for f in preprocessing_pipeline:
        s = f(s)
    print(s)

In [ ]:
[pair for pair in sorted(similar_pairs, key=lambda x: -x[1])]

In [ ]:
save checkpoint

### New test 100

In [9]:
ckpt_path = f'checkpoints/100'
model = LSHModel(
    k=4,
    threshold=0.2,
    num_hashes=200,
    shingle_hash_bits=18,
    track_shingles=True,
    checkpoint_path=ckpt_path
)
model = train_model(
    model=model, 
    data_path=DATA_PATH,
    num_docs=10000,
    verbose=True,
    filtering_pipeline=filtering_pipeline,
    preprocessing_pipeline=preprocessing_pipeline
)

Adding documents to model:   0%|          | 0/10000 [00:00<?, ?it/s]

Reading file e:\datasets\ukraine\0401_UkraineCombinedTweetsDeduped.csv.gzip
Reading file e:\datasets\ukraine\0402_UkraineCombinedTweetsDeduped.csv.gzip
Reading file e:\datasets\ukraine\0403_UkraineCombinedTweetsDeduped.csv.gzip
Reading file e:\datasets\ukraine\0404_UkraineCombinedTweetsDeduped.csv.gzip
Reading file e:\datasets\ukraine\0405_UkraineCombinedTweetsDeduped.csv.gzip
Reading file e:\datasets\ukraine\0406_UkraineCombinedTweetsDeduped.csv.gzip


KeyboardInterrupt: 

In [ ]:
model.docs_dict

In [ ]:
pairs = model.get_similar_pairs()

In [ ]:
model.docs_dict[1]

In [ ]:
model.docs_dict[59]

In [ ]:
get_text((1,2,3,4,217, 974), DATA_PATH, filtering_pipeline=filtering_pipeline)

In [ ]:
sorted(pairs, key=lambda x: -x[1])

### MPNet embeddings

CHECK ONLY ON PAIRS WITH < 0.9 or 0.8

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'sentence-transformers/all-mpnet-base-v2'
)
mpnet = AutoModel.from_pretrained(
    'sentence-transformers/all-mpnet-base-v2'
).to(device)

In [ ]:
mpnet_preprocessing = [
    replace_chars,
    str.lower,
    normalize_white_space
]

In [ ]:
idx_ls = np.unique(
    np.array(
        [
            list(pair)
            for pair, _ in similar_pairs
        ] 
    ).flatten()
)
text_dict = dict(get_text(idx_ls, DATA_PATH))

preprocessed_texts = []
for text in text_dict.values():
    for f in mpnet_preprocessing:
        text = f(text)
    preprocessed_texts.append(text)

encoded_input = tokenizer(
    preprocessed_texts, 
    padding='max_length', 
    truncation=True, 
    return_tensors='pt'
).to(device)

In [ ]:
with torch.no_grad():
    model_output = model(**encoded_input)

embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

embeddings_dict = {
    key: val
    for key, val in zip(text_dict.keys(), embeddings)
}

In [ ]:
lsh_sims = []
mpnet_sims = []

for ((x_idx, y_idx), lsh_sim) in similar_pairs:
    lsh_sims.append(lsh_sim)
    mpnet_sims.append(
        torch_cosine_similarity(
            embeddings_dict[x_idx],
            embeddings_dict[y_idx],
        )
    )

### Compare rankings

In [ ]:
kendalltau(lsh_sims, mpnet_sims)

In [ ]:
spearmanr(lsh_sims, mpnet_sims)

In [ ]:
ckpt_path = f'checkpoints/test/test1'
model = LSHModel(
    k=5,
    threshold=0.1,
    num_hashes=100,
    shingle_hash_bits=16,
    track_shingles=True,
    checkpoint_path=ckpt_path
)

num_docs = 100

files = []
data_path = DATA_PATH

for name in os.listdir(data_path):
    full_path = os.path.join(data_path, name)
    if os.path.isfile(full_path):
        files.append(full_path)

duplicates = 0
count = num_docs

with tqdm(
    total=num_docs,
    desc='Adding documents to model',
    leave=False
) as pbar:
    for file in files:
        if count == 0:
            break
        
        print(f'Reading file {file}')
        df = pd.read_csv(
            file, 
            compression='gzip', 
            index_col=0,
            encoding='utf-8', 
            quoting=csv.QUOTE_ALL,
            low_memory=False
        )

        df = df[df['language'] == 'en']

        for filter_f in filtering_pipeline:
            df['text'] = df['text'].apply(filter_f)

        df_unique = df.drop_duplicates(subset=['text'])
        duplicates += len(df) - len(df_unique)

In [ ]:
# List of Tuples
matrix = [('22', '34', '23'),
         ('33', '31', '11'),
         ('44', '16', '21'),
         ('55', '32', '22'),
         ('66', '33', '27'),
         ('77', '35', '11')
         ]
# Create a DataFrame object
dfObj = pd.DataFrame(matrix, columns=list('xyz'), index=list('abcdef'))

In [ ]:
dfObj

In [ ]:
dfObj['z'] = dfObj['z'].apply(remove_https)
dfObj

In [ ]:
remove_https('ababss')

In [ ]:
matrix = [('22', '34', '23'),
         ('33', '31', '11'),
         ('44', '16', '21'),
         ('22', '34', '23'),
         ('66', '33', '27'),
         ('22', '34', '23')
         ]
# Create a DataFrame object
dd = pd.DataFrame(matrix, columns=list('xyz'), index=list('abcdef'))
dd

In [ ]:
dd1 = dd.drop_duplicates()
dd1

In [ ]:
dd.loc[dd1.index]

In [ ]:
s = '@mr_589_ putin is a cancer to the world. @russia need to overthrow him asap. stand with @ukraine! #standwithukraine'

In [ ]:
def remove_handles(doc: str) -> str:
    return re.sub(r'@\w+', '', doc)

In [ ]:
remove_handles(s)

In [ ]:
def remove_short(n: int):
    def inner_f(doc: str):
        if len(doc) < n:
            return ''
        else:
            return doc

    return inner_f

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [33]:
remover = get_stopwords_remover(stops)

In [37]:
s = "Hey, #Ukraine is not just angry ( understatement!) At @KremlinRussia_E . It's angry at the rest of the world as well for allowing this war to happen. \n\nAnd you know what? It has every right to be so.\n\nI simply understand it. \n\nThis war can be stopped within 5 minutes."
ss = remover(s.lower())
ss

'hey, #ukraine angry ( understatement!) @kremlinrussia_e . angry rest world well allowing war happen. know what? every right so. simply understand it. war stopped within 5 minutes.'

In [38]:
lemmatizer = get_lemmatizer(nlp)

In [39]:
lemmatizer(ss)

'hey angry understatement @kremlinrussia_e angry rest world allow war happen know right simply understand war stop minute'

In [27]:
len(stops)

179